In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp client

<IPython.core.display.Javascript object>

In [ ]:
# export

import os
import math
import time
import httpx
import asyncio
import aiohttp

from pathlib import Path
from multiprocessing import Pool
from multiprocessing import set_start_method

from will_it_saturate.core import Benchmark, BenchmarkServer

try:
    set_start_method("fork")
except RuntimeError:
    pass

In [ ]:
print(os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"])

YES


# Caveats

On macOS increase open file limit with:

```
ulimit -n 2048
```

Before starting the fastAPI Server with:

```
uvicorn will_it_saturate.main:app --reload
```

In [ ]:
# export


from will_it_saturate.core import BenchmarkClient


class HttpxClient(BenchmarkClient):
    async def measure_server(self, benchmark_row):
        print("measure server")
        urls = [bf.url for bf in benchmark_row.files]
        print(urls[0])
        max_connections = min(benchmark_row.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        print("done")
        return elapsed, responses

    def measure_in_new_process(self, benchmark_row):
        print("new process")
        elapsed, responses = asyncio.run(self.measure_server(benchmark_row))
        self.verify_checksums(benchmark_row.files, responses)
        return elapsed

    def measure(self, benchmark_row):
        print("measure")
        with Pool(1) as p:
            [result] = p.map(self.measure_in_new_process, [benchmark_row])
        return result


def run_httpx():
    byte = 8
    gigabit = 10 ** 9
    bandwidth = gigabit / byte

    # file_sizes = [10 ** 7, 10 ** 6]
    # file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
    file_sizes = [10 ** 7]

    benchmark = Benchmark(
        bandwidth=bandwidth,
        duration=3,
        file_sizes=file_sizes,
        servers=[BenchmarkServer(name="uvicorn")],
        clients=[HttpxClient(name="httpx")],
    )
    benchmark.create_rows()
    benchmark.run()
    print(benchmark.results_frame)

#    client = HttpxClient()
#    result = client.measure(benchmark.rows[0])
#    print(result)

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte


# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]

benchmark = Benchmark(
    bandwidth=bandwidth,
    duration=3,
    file_sizes=file_sizes,
    servers=[BenchmarkServer(name="uvicorn")],
    clients=[HttpxClient(name="httpx")],
)
benchmark.create_rows()

In [ ]:
%%time

benchmark.run()

measure
new process
measure server
http://localhost:8000/data/10000000_3_125000000/0
done
measure
new process
measure server
http://localhost:8000/data/1000000_3_125000000/0
done
measure
new process
measure server
http://localhost:8000/data/100000_3_125000000/0
done
CPU times: user 21.7 ms, sys: 33.9 ms, total: 55.6 ms
Wall time: 16.7 s


In [ ]:
benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,uvicorn,httpx,10000000,3.279600,375000000,9.54MB,1.143432e+08,109.05MB
1,uvicorn,httpx,1000000,4.131801,375000000,976.56KB,9.075945e+07,86.55MB
2,uvicorn,httpx,100000,6.791448,375000000,97.66KB,5.521650e+07,52.66MB


In [ ]:
client = HttpxClient()
result = client.measure(benchmark.rows[0])
print(result)

measure
new process
measure server
http://localhost:8000/data/10000000_3_125000000/0
done
3.2253238749999866


In [ ]:
benchmark.rows

[]

<IPython.core.display.Javascript object>

In [ ]:
# hide

from will_it_saturate.core import BenchmarkClient


class HttpxClient(BenchmarkClient):
    async def measure_br(self, br):
        urls = [bf.url for bf in br.files]
        # httpx breaks on more than 100 parallel connections
        max_connections = min(br.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        return elapsed, responses

    async def measure_async(self, br):
        elapsed, responses = await self.measure_br(br)
        print(elapsed, len(responses))
        self.verify_checksums(br.files, responses)
        return elapsed

    def measure_benchmark_row(self, br):
        loop = asyncio.get_running_loop()
        task = loop.create_task(self.measure_async(br))
        task.add_done_callback(
            lambda t: print(
                f"Task done: " f"{t.result()=} << return val of main()"  # optional
            )
        )  # optional (using py38)

        # asyncio.run(self.measure_async(br))
        # return 2.0


#         urls = [bf.url for bf in br.files]
#         elapsed, responses = await self.measure_urls(urls)
#         self.verify_checksums(br.files, responses)

<IPython.core.display.Javascript object>

In [ ]:
client = HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

In [ ]:
benchmark.clients.append(client)

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.core import BenchmarkServer

<IPython.core.display.Javascript object>

In [ ]:
benchmark.servers.append(BenchmarkServer(name="fastAPI"))

<IPython.core.display.Javascript object>

In [ ]:
benchmark.run()

ValidationError: 1 validation error for BenchmarkResult
elapsed
  none is not an allowed value (type=type_error.none.not_allowed)

<IPython.core.display.Javascript object>

4.2216322360000005 38
Task done: t.result()=4.2216322360000005 << return val of main()


In [ ]:
# hide

from will_it_saturate.core import BenchmarkClient


class HttpxClient(BenchmarkClient):
    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        # httpx breaks on more than 100 parallel connections
        max_connections = min(br.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        c
        return elapsed

<IPython.core.display.Javascript object>

In [ ]:
bclient = HttpxClient(benchmark, "fastAPI", "httpx")

<IPython.core.display.Javascript object>

In [ ]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
bclient.show_results()

Filesize  Transferred for server fastAPI with client httpx
9.54MB   90.34MB/s
976.56KB   87.31MB/s
97.66KB   38.52MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI + httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI + httpx: 
---------
File size: 9.54MB
59.63MB/s
---------
File size: 976.56KB
48.34MB/s
---------
File size: 97.66KB
11.57MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Filesize  Transferred for ")
file_size, file_unit = "97.66", "KB"
file_string = f"{file_size}{file_unit}"
transferred_per_second, transferred_unit = "11.57", "MB"
transferred_string = f"{transferred_per_second}{transferred_unit}/s"
line = f"{file_string} {transferred_string:>10s}"
print(line)

Filesize  Transferred for 
97.66KB  11.57MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Filesize  Transferred for ")
file_size, file_unit = "976.56", "KB"
file_string = f"{file_size}{file_unit}"
transferred_per_second, transferred_unit = "11.57", "MB"
transferred_string = f"{transferred_per_second}{transferred_unit}/s"
line = f"{file_string} {transferred_string:>10s}"
print(line)

Filesize  Transferred for 
976.56KB  11.57MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI + httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI + httpx: 
---------
File size: 9.54MB
46.68MB/s
---------
File size: 976.56KB
42.95MB/s
---------
File size: 97.66KB
26.27MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx: 
---------
File size: 9.54MB
55.56MB/s
---------
File size: 976.56KB
49.69MB/s
---------
File size: 97.66KB
20.27MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with httpx: 
---------
File size: 9.54MB
62.09MB/s
---------
File size: 976.56KB
74.07MB/s
---------
File size: 97.66KB
30.94MB/s


<IPython.core.display.Javascript object>

# aiohttp

In [ ]:
# hide


class AioHttpResponse:
    def __init__(self, url, content):
        self.url = url
        self.content = content


class AioHttpClient(BaseClient):
    async def fetch_page(self, session, url):
        async with session.get(url) as response:
            content = await response.read()
            return AioHttpResponse(url, content)

    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        max_connections = min(br.number_of_connections, 200)
        # max_connections = br.number_of_connections
        conn = aiohttp.TCPConnector(limit=max_connections)
        responses = []
        start = time.perf_counter()
        async with aiohttp.ClientSession(connector=conn) as session:
            tasks = [asyncio.create_task(self.fetch_page(session, url)) for url in urls]
            responses = await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        self.responses = responses
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [ ]:
bclient = AioHttpClient(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

61.16 MB
54.44 MB
31.9 MB


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI with aiohttp: 
---------
File size: 9.54MB
49.37MB/s
---------
File size: 976.56KB
52.43MB/s
---------
File size: 97.66KB
35.38MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with aiohttp: 
---------
File size: 9.54MB
60.81MB/s
---------
File size: 976.56KB
56.63MB/s
---------
File size: 97.66KB
40.02MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with aiohttp: 
---------
File size: 9.54MB
80.27MB/s
---------
File size: 976.56KB
71.1MB/s
---------
File size: 97.66KB
56.49MB/s


<IPython.core.display.Javascript object>

# Export

In [ ]:
from nbdev.export import snotebook2script

notebook2script()

ImportError: cannot import name 'snotebook2script' from 'nbdev.export' (/Users/jochen/.pyenv/versions/3.9.1/envs/wis/lib/python3.9/site-packages/nbdev/export.py)